In [ ]:
!pip install psycopg2

In [ ]:
import psycopg2

con = psycopg2.connect(
        dbname="akarve",
        user="postgres",
        password="postgres",
        host="localhost",
        port=5436,
    )
con

In [ ]:
cursor = con.cursor()

In [ ]:
from psycopg2 import sql

In [ ]:
from collections import OrderedDict

In [ ]:
system_id_metadata = {
    'datatype_csv': 'bigint',
    'datatype_table': 'bigint',
    'field_name': 'system_id',
    'encrypt': False
}
cluster_id_metadata = {
    'datatype_csv': 'bigint',
    'datatype_table': 'bigint',
    'field_name': 'cluster_id',
    'encrypt': False
}
id_metadata = {
    'datatype_csv': 'bigint',
    'datatype_table': 'bigint',
    'field_name': 'id',
    'encrypt': False
}
created_metadata = {
    'datatype_csv': 'timestamp',
    'datatype_table': 'timestamp',
    'field_name': 'created',
    'encrypt': False
}
uuid_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'varchar',
    'field_name': 'uuid',
    'encrypt': False
}
parent_uuid_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'varchar',
    'field_name': 'parent_uuid',
    'encrypt': False
}

event_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'varchar',
    'field_name': 'event',
    'encrypt': False
}
task_action_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'bigint REFERENCES task_action_tmp(id)',
    'field_name': 'task_action',
    'encrypt': True
}
failed_metadata = {
    'datatype_csv': 'boolean',
    'datatype_table': 'boolean',
    'field_name': 'failed',
    'encrypt': False
}
changed_metadata = {
    'datatype_csv': 'boolean',
    'datatype_table': 'boolean',
    'field_name': 'changed',
    'encrypt': False
}
playbook_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'bigint REFERENCES playbook_tmp(id)',
    'field_name': 'playbook',
    'encrypt': True
}
play_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'bigint REFERENCES play_tmp(id)',
    'field_name': 'play',
    'encrypt': True
}
task_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'bigint REFERENCES task_tmp(id)',
    'field_name': 'task',
    'encrypt': True
}
role_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'bigint REFERENCES role_tmp(id)',
    'field_name': 'role',
    'encrypt': True
}
job_id = {
    'datatype_csv': 'bigint',
    'datatype_table': 'bigint',
    'field_name': 'job_id',
    'encrypt': False
}
host_id_metadata = {
    'datatype_csv': 'bigint',
    'datatype_table': 'bigint',
    'field_name': 'host_id',
    'encrypt': False
}
host_name_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'bigint REFERENCES host_tmp(id)',
    'field_name': 'host',
    'encrypt': True
}
start_metadata = {
    'datatype_csv': 'timestamp',
    'datatype_table': 'timestamp',
    'field_name': 'start',
    'encrypt': False
}
end_metadata = {
    'datatype_csv': 'timestamp',
    'datatype_table': 'timestamp',
    'field_name': 'end',
    'encrypt': False
}
duration_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'float',
    'field_name': 'duration',
    'encrypt': False
}
warnings_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'varchar',
    'field_name': 'warnings',
    'encrypt': True
}
deprecations_metadata = {
    'datatype_csv': 'varchar',
    'datatype_table': 'varchar',
    'field_name': 'deprecations',
    'encrypt': True
}

In [ ]:
job_events_tmp_dict = {
   'system_id': system_id_metadata,
   'cluster_id': cluster_id_metadata,
   'id': id_metadata,
   'created': created_metadata,
   'uuid': uuid_metadata,
   'parent_uuid': parent_uuid_metadata,
   'event': event_metadata,
   'task_action': task_action_metadata,
   'failed': failed_metadata,
   'changed': changed_metadata,
   'playbook': playbook_metadata,
   'play': play_metadata,
   'task': task_metadata,
   'role': role_metadata,
   'job_id': job_id,
   'host_id': host_id_metadata,
   'host_name': host_name_metadata,
   'start': start_metadata,
   'end': end_metadata,
   'duration': duration_metadata,
   'warnings': warnings_metadata,
   'deprecations': deprecations_metadata
}

jo = OrderedDict(job_events_tmp_dict.items())
jo


In [ ]:
for item in jo.items():
    print(f"{item[0]} {type(item[1])}")

In [ ]:
whitelist_varchar_fields = {
 'task_action': 'task_action',
 'playbook': 'playbook',
 'play': 'play',
 'task': 'task',
 'role': 'role',
 'host_name': 'host',
 'warnings': 'warnings',
 'deprecations': 'deprecations'
}
varchar_fields = ([whitelist_varchar_fields[field] for field, field_type in jo.items() if field in whitelist_varchar_fields.keys()])

In [ ]:
def tmp_table(table_name, datatype):
#     column_types = [f"{sql.Identifier(c[0]).as_string(con)} {c[1]}" for c in jo.items()]

    model_query = sql.SQL('''
        DROP TABLE IF EXISTS {table} CASCADE;
        CREATE TABLE {table}({fields_with_types});
    ''') 
#     fields_with_types = sql.SQL(",".join([f"{sql.Identifier(c[0]).as_string(con)} {c[1]} REFERENCES {whitelist_varchar_fields.get(c[0], c[0])}_tmp(id)" for c in jo.items()]))
    fields_with_types = sql.SQL(",".join([f"{sql.Identifier(c[0]).as_string(con)} {c[1][datatype]}" for c in jo.items()]))

    return model_query.format(table=sql.Identifier(f"{table_name}"), fields_with_types=fields_with_types)


In [ ]:
query = tmp_table("job_events_tmp", "datatype_csv")
query

In [ ]:
query = tmp_table("job_events_fk_tmp", "datatype_table")
query

In [ ]:
column_types = [f"{sql.Identifier(c[0]).as_string(con)} {c[1]}" for c in jo.items()]

table_name = sql.Identifier("job_events_tmp")

model_query = sql.SQL('''
    DROP TABLE IF EXISTS {table_name} CASCADE;
    CREATE TABLE {table_name}({fields_with_types});
''') 
fields_with_types = sql.SQL(",".join([f"{sql.Identifier(c[0]).as_string(con)} {c[1]}" for c in jo.items()]))

query = model_query.format(table_name=table_name, fields_with_types=fields_with_types)
query

In [ ]:
cursor.execute(query)

In [ ]:
con.commit()

In [ ]:
varchar_tmp_table_query = '''
    DROP TABLE IF EXISTS {varchar_tmp_table} CASCADE;
    CREATE TABLE {varchar_tmp_table} (
        id serial primary key,
        name varchar NOT NULL UNIQUE,
        hash varchar,
        CHECK (char_length(name) > 0)
    );
    
    DROP TABLE IF EXISTS {varchar_tmp_map_table} CASCADE;
    CREATE TABLE {varchar_tmp_map_table} (
        id bigint,
        tmp_id bigint
    );
'''

master_varchar_tmp_table_query = ''
for f in jo.items():
    if f[1]['encrypt']:
        q = varchar_tmp_table_query.format(varchar_tmp_table=f"{f[1]['field_name']}_tmp", varchar_tmp_map_table=f"{f[1]['field_name']}_map_tmp")
        master_varchar_tmp_table_query += q
    

In [ ]:
cursor.execute(master_varchar_tmp_table_query)

In [ ]:
con.commit()